In [1]:
import pandas as pd
import numpy as np
import os
import ruptures as rpt

import sys
sys.path.append(os.path.abspath('../OFVMNET'))

from OFProcessor import OpticalFlowProcessor
from sklearn.model_selection import train_test_split

## Create Train Test Split

In [7]:
path = r"/Users/scottmerrill/Documents/UNC/MultiModal/VMR/Youtube8m"
filenames = os.listdir(path + '/video')

In [8]:
valid_files = []
for file_name in filenames:
    try:
        video_data = np.load(os.path.join(path, 'video', file_name))

        processor = OpticalFlowProcessor(method='video', segments=20, min_frames=5, window_size=20)
        of_video = processor._compute_flow(video_data, _)
        change_points = processor._optical_flow_segments(of_video,  max_seq_len=100)
        differences = [change_points[i+1] - change_points[i] for i in range(len(change_points)-1)]

        # if this is longer than our sequence length it's not a valid file for training
        if max(differences) > 100:
            continue
        else:
            valid_files.append(file_name)
    except Exception as e:
        # note this can fail for a number of reasons
        # since we specify a minimum segments of 10 and at least 10 second clips, 
        print(e)

In [12]:
files = pd.DataFrame(valid_files, columns=['filename'])

In [13]:
train_df, test_df = train_test_split(files, test_size=0.2, random_state=42)

In [14]:
train_df.to_csv(path+'/train.csv', index=False)
test_df.to_csv(path+'/test.csv', index=False)